<a href="https://colab.research.google.com/github/mmdedavoodi/cats-dogs-resnet50-transfer-learning/blob/main/Cats_Dog_CNN_TransferLearning_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1IcAf8TmM2T7HB_jvb-cKela94_ZIPuqP

In [ ]:
!unzip /content/cats_and_dogs_filtered.zip

In [ ]:
import random
import matplotlib.pyplot as plt
import torch
import cv2
import torchvision.transforms as t
import numpy as np
from glob import glob
from torchvision.datasets import DatasetFolder
from torch import nn
from tqdm import tqdm_notebook as tqdm
import torchvision
from google.colab.patches import cv2_imshow

In [ ]:
images_file_path_train = glob('/content/cats_and_dogs_filtered/train/*/*.jpg')
images_file_path_test = glob('/content/cats_and_dogs_filtered/validation/*/*.jpg')

In [ ]:
images_file_path_train[0].split("/")[-2]

In [ ]:
len(images_file_path_train), len(images_file_path_test)

## DataSet

In [ ]:
class DataSetCatDog(torch.utils.data.Dataset):
  def __init__(self, images_file_path, transform=None):
    self.images_file_path = images_file_path
    random.shuffle(self.images_file_path)
    self.transform = transform

  def __len__(self):
    return len(self.images_file_path)

  def __getitem__(self, idx):
    image_path = self.images_file_path[idx]
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    label = image_path.split("/")[-2]
    if label == "cats":
      label = 1
    elif label == "dogs":
      label = 0
    if self.transform:
      image = self.transform(image)

    return image , label

In [ ]:
from torchvision import models

In [ ]:
train_transforms = t.Compose([
    t.ToTensor(),
    # t.Resize((224,224)),
    t.RandomHorizontalFlip(p=0.5),
    t.RandomRotation(degrees=20),
    t.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6),
    models.ResNet50_Weights.IMAGENET1K_V2.transforms()
])

test_transforms = t.Compose([
    t.ToTensor(),
    models.ResNet50_Weights.IMAGENET1K_V2.transforms()
])


In [ ]:
models.ResNet50_Weights.IMAGENET1K_V2.transforms()

In [ ]:
DataSetTrain = DataSetCatDog(images_file_path_train, transform=train_transforms)
DataSetTest = DataSetCatDog(images_file_path_test, transform=test_transforms)

In [ ]:
plt.imshow(DataSetTrain[0][0].permute(1,2,0))
print(DataSetTrain[0][1])
DataSetTrain[0][0].shape


## DataLoader

In [ ]:
Dataloader_Train = torch.utils.data.DataLoader(DataSetTrain, batch_size=64, shuffle=True , num_workers=2 , prefetch_factor=2)
Dataloader_Test = torch.utils.data.DataLoader(DataSetTest, batch_size=64, shuffle=False , num_workers=2 , prefetch_factor=2)

## Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

In [ ]:
model.fc = nn.Identity()
model.to(device)

In [ ]:
model.layer4[0].conv3.weight[0].shape

In [ ]:
plt.imshow(model.conv1.weight[-3].detach().cpu().permute(1,2,0).numpy())

In [ ]:
for image , label in Dataloader_Train:
  break


In [ ]:
image.shape

In [ ]:
label

In [ ]:
label.argsort()

In [ ]:
with torch.no_grad():
    output = model(image.to(device))

In [ ]:
output.shape


In [ ]:
from torchvision.models import ResNet50_Weights
idx_to_label = ResNet50_Weights.DEFAULT.meta['categories']
idx_to_label

In [ ]:
plt.figure(figsize=(90, 20))
plt.imshow(output.cpu()[label.argsort()])

## Extract Features

In [ ]:
## Train Phase

pbar = tqdm(Dataloader_Train)
features_train = []
labels_train = []
for image , label in pbar:
  with torch.no_grad():
    image , label = image.to(device) , label.to(device)
    output = model(image)
    features_train.append(output.cpu().numpy())
    labels_train.append(label.cpu().numpy())

In [ ]:
features_train = np.vstack(features_train)
labels_train = np.hstack(labels_train)

In [ ]:
features_train.shape , labels_train.shape

In [ ]:
## Test Phase

pbar = tqdm(Dataloader_Test)
features_test = []
labels_test = []
for image , label in pbar:
  with torch.no_grad():
    image , label = image.to(device) , label.to(device)
    output = model(image)
    features_test.append(output.cpu().numpy())
    labels_test.append(label.cpu().numpy())

In [ ]:
features_test = np.vstack(features_test)
labels_test = np.hstack(labels_test)

In [ ]:
features_test.shape , labels_test.shape

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report , accuracy_score

In [ ]:
model_svm = SVC(kernel = "rbf")
model_svm.fit(features_train, labels_train)

In [ ]:
label_pred = model_svm.predict(features_test)

In [ ]:
accuracy_score(labels_test, label_pred) * 100

In [ ]:
print(classification_report(labels_test, label_pred))

In [ ]:
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay

In [ ]:
c = confusion_matrix(labels_test, label_pred)
dis = ConfusionMatrixDisplay(c , display_labels=["Dog" , "Cat"] )
dis.plot()

In [ ]:
model

## Fine Tuning

In [ ]:
class BackBoneCatDog(nn.Module):
  def __init__(self):
    super().__init__()
    self.feature_extractor = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    self.feature_extractor.fc = nn.Identity()
    self.cls_head = nn.Linear(2048, 1)

    self.lock = False

  def forward(self, x):
    if self.lock:
      with torch.no_grad():
        z = self.feature_extractor(x)
    else:
      z = self.feature_extractor(x)
    z = self.cls_head(z)
    return z

  def freeze(self):
    self.lock = True
    self.feature_extractor.requires_grad_(False)

  def unfreeze(self):
    self.lock = False
    self.feature_extractor.requires_grad_(True)


In [ ]:
model = BackBoneCatDog()

In [ ]:
model

In [ ]:
model.to(device)
model.freeze()

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.cls_head.parameters(), lr=0.001)

In [ ]:
def train_Phase(model , dataloader , optimizer , loss_function , device):
  model.train()
  train_phase_loss = 0
  train_phase_acc = 0
  sample = len(dataloader.dataset)
  pbar = tqdm(dataloader , desc = "Training")
  for image , label in pbar:
    image , label = image.to(device) , label.to(device)
    label = label.view(-1, 1).float()
    output = model(image)
    output_sigmoid = torch.sigmoid(output)
    loss_value = loss_function(output , label)
    loss_value.backward()
    pbar.set_postfix_str(f"Loss = {loss_value:.3f}")
    optimizer.step()
    optimizer.zero_grad()

    train_phase_loss += loss_value.item() * len(label)
    train_phase_acc += (output_sigmoid.round() == label).sum().item()

  train_phase_loss /= sample
  train_phase_acc /= sample
  return train_phase_loss , train_phase_acc

In [ ]:
def test_Phase(model , dataloader , loss_function , device):
  model.eval()
  test_phase_loss = 0
  test_phase_acc = 0
  pbar = tqdm(dataloader , desc = "Testing")
  sample = len(dataloader.dataset)

  for image, label in pbar:
    image , label = image.to(device) , label.to(device)
    label = label.view(-1, 1).float()
    with torch.no_grad():
      output = model(image)
      output_sigmoid = torch.sigmoid(output)
      loss_value = loss_function(output , label)
      pbar.set_postfix_str(f"Loss = {loss_value:.3f}")

      test_phase_loss += loss_value.item() * len(label)
      test_phase_acc += (output_sigmoid.round() == label).sum().item()

  test_phase_loss /= sample
  test_phase_acc /= sample

  return test_phase_loss , test_phase_acc

In [ ]:
def run_epoch(model , train_dataloader , test_dataloader , optimizer , loss_function , device):
  train_loss , train_acc = train_Phase(model , train_dataloader , optimizer , loss_function , device)
  test_loss , test_acc = test_Phase(model , test_dataloader , loss_function , device)
  return train_loss , train_acc , test_loss , test_acc

In [ ]:
def train_model(model , train_dataloader , test_dataloader , optimizer , loss_function , device , epochs = 10):
  train_loss = []
  train_acc = []
  test_loss = []
  test_acc = []
  for epoch in range(epochs):
    try:
      train_phase_loss , train_phase_acc , test_phase_loss , test_phase_acc = run_epoch(model , train_dataloader , test_dataloader , optimizer , loss_function , device)
      print(f"Epoch {epoch + 1}: train loss {train_phase_loss:4f}, train acc {train_phase_acc * 100:2f} | test loss {test_phase_loss:4f}, test acc {test_phase_acc * 100:2f}")
      train_loss.append(train_phase_loss)
      train_acc.append(train_phase_acc)
      test_loss.append(test_phase_loss)
      test_acc.append(test_phase_acc)
    except KeyboardInterrupt:
      break
  return train_loss , train_acc , test_loss , test_acc

In [ ]:
def draw_plot(train_loss , train_acc , test_loss , test_acc):
  plt.figure(figsize=(10, 4))
  plt.subplot(1, 2, 1)
  plt.plot(train_loss, label="Train")
  plt.plot(test_loss, label="Test")
  plt.legend()
  plt.grid()
  plt.ylabel('Loss')
  plt.xlabel('Epoch')

  plt.subplot(1, 2, 2)
  plt.plot(train_acc, label="Train")
  plt.plot(test_acc, label="Test")
  plt.legend()
  plt.grid()
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')

  return plt

In [ ]:
train_loss , train_acc , test_loss , test_acc = train_model(model , Dataloader_Train , Dataloader_Test , optimizer , loss_function , device , 10)

In [ ]:
draw_plot(train_loss , train_acc , test_loss , test_acc)

In [ ]:
i += 1
res = model(DataSetTest[i][0].to(device).unsqueeze(0)).sigmoid().round().item()
plt.imshow(DataSetTest[i][0].permute(1,2,0))
print(res)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)
model.unfreeze()

In [ ]:
train_loss , train_acc , test_loss , test_acc = train_model(model , Dataloader_Train , Dataloader_Test , optimizer , loss_function , device , 5)

In [ ]:
draw_plot(train_loss , train_acc , test_loss , test_acc)